# Análisis @robados_chile

En este notebook analizaremos los tweets de la cuenta de Twitter @robados_chile que muestra mensajes acerca de autos robados en Chile. Intentaremos obtener los tópicos más frecuentes en los tweets y ver cómo se comportan los mensajes. En esta ocasión se realizará el Topic Learning con 1000 iteraciones.

## 1. Indexar las palabras en los documentos

In [1]:
%%bash

input_dir=./BTM/sample-data/
output_dir=./BTM/output-tweets-robados_chile/
model_dir=${output_dir}model/
mkdir -p $output_dir/model 

# the input docs for training
doc_pt=${input_dir}tweets-robados_chile-filtered.txt

echo "=============== Index Docs ============="
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
# vocabulary file
voca_pt=${output_dir}voca.txt
python3 ./BTM/script/indexDocs.py $doc_pt $dwid_pt $voca_pt

=============== Index Docs =============
index file: ./BTM/sample-data/tweets-robados_chile-filtered.txt
write file: ./BTM/output-tweets-robados_chile/doc_wids.txt
n(w)=5369
write:./BTM/output-tweets-robados_chile/voca.txt


## 2. Topic Learning

In [2]:
%%bash

# number of topics
K=25

alpha=`echo "scale=3;50/$K"|bc`
beta=0.005
niter=1000
save_step=501

input_dir=./BTM/sample-data/
output_dir=./BTM/output-tweets-robados_chile/
model_dir=${output_dir}model/
# the input docs for training
doc_pt=${input_dir}tweets-robados_chile-filtered.txt
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
# vocabulary file
voca_pt=${output_dir}voca.txt

## learning parameters p(z) and p(w|z)
echo "=============== Topic Learning ============="
W=`wc -l < $voca_pt` # vocabulary size
make -C ./BTM/src
echo "./BTM/src/btm est $K $W $alpha $beta $niter $save_step $dwid_pt $model_dir"
./BTM/src/btm est $K $W $alpha $beta $niter $save_step $dwid_pt $model_dir

=============== Topic Learning =============
make: Nothing to be done for `all'.
./BTM/src/btm est 25     5369 2.000 0.005 1000 501 ./BTM/output-tweets-robados_chile/doc_wids.txt ./BTM/output-tweets-robados_chile/model/
Run BTM, K=25, W=5369, alpha=2, beta=0.005, n_iter=1000, save_step=501 ====
load docs: ./BTM/output-tweets-robados_chile/doc_wids.txt
Begin iteration
iter 501/1000
write p(z): ./BTM/output-tweets-robados_chile/model/k25.pz
write p(w|z): ./BTM/output-tweets-robados_chile/model/k25.pw_z
iter 1000/1000
write p(z): ./BTM/output-tweets-robados_chile/model/k25.pz
write p(w|z): ./BTM/output-tweets-robados_chile/model/k25.pw_z
cost 251.056621s


## 3. Inferir las proporciones de tópicos para cada uno de los documentos

In [3]:
%%bash

# number of topics
K=25

output_dir=./BTM/output-tweets-robados_chile/
# docs after indexing
dwid_pt=${output_dir}doc_wids.txt
model_dir=${output_dir}model/

## infer p(z|d) for each doc
echo "================ Infer P(z|d)==============="
echo "./BTM/src/btm inf sum_b $K $dwid_pt $model_dir"
./BTM/src/btm inf sum_b $K $dwid_pt $model_dir

================ Infer P(z|d)===============
./BTM/src/btm inf sum_b 25 ./BTM/output-tweets-robados_chile/doc_wids.txt ./BTM/output-tweets-robados_chile/model/
Run inference:K=25, type sum_b ====
load p(z):./BTM/output-tweets-robados_chile/model/k25.pz
load p(w|z):./BTM/output-tweets-robados_chile/model/k25.pw_z
n(z)=25, n(w)=5369
Infer p(z|d) for docs in: ./BTM/output-tweets-robados_chile/doc_wids.txt
write p(z|d): ./BTM/output-tweets-robados_chile/model/k25.pz_d


## 4. Presentar las palabras más frecuentes por cada tópico y sus proporciones en la colección

In [4]:
%%bash

K=25   # number of topics

output_dir=./BTM/output-tweets-robados_chile/
model_dir=${output_dir}model/

# vocabulary file
voca_pt=${output_dir}voca.txt

## output top words of each topic
echo "================ Topic Display ============="
python3 ./BTM/script/topicDisplay.py $model_dir $K $voca_pt

================ Topic Display =============
K:25, n(W):5369
p(z)		Top words
0.095701	133:0.078359 pdi:0.074230 carab:0.073454 info:0.069052 #robado:0.067341 patente:0.065014 134:0.062619 color:0.038347 dm:0.037138 modelo:0.029678
0.076624	patente:0.070878 #robado:0.068542 info:0.063328 ref:0.057488 foto:0.056691 mano:0.051762 armada:0.049369 comuna:0.042219 carab:0.040253 kia:0.022192
0.065908	#robado:0.081101 @autorobado_cl:0.080373 patente:0.074014 @portonazoen:0.070703 foto:0.058748 ref:0.042952 @rnesantiago:0.038912 @grupowurtlitzer:0.030533 @chileenpaz:0.028977 @dondatos:0.027619
0.064327	#robado:0.073492 patente:0.072983 info:0.063211 foto:0.053643 ref:0.052964 armada:0.048994 mano:0.045534 carab:0.036610 comuna:0.034846 nissan:0.021240
0.062680	info:0.071138 patente:0.069954 #robado:0.068701 carab:0.049550 foto:0.037850 comuna:0.036527 toyota:0.035447 133:0.035134 ref:0.031756 color:0.024514
0.060749	patente:0.074763 #robado:0.071457 info:0.062835 foto:0.059171 hyundai:0.051303

De los tópicos encontrados notamos que varios de ellos hacen referencia a marcas de autos comúnmente robados. Encontramos marcas como Kia, Nissan, Hyundai, Toyota. Por otra parte notamos que los tópicos asociados a robos de autos y a portonazos, por ejemplo, se presentan en conjunto de la comuna de Vitacura. Por otra parte vemos que tópicos asociados a recuperación de vehículos robados están asociados a la comuna de Quilicura, por ejemplo.